In [1]:
import cv2
import numpy as np
from realsense_depth import *

In [ ]:
video = cv2.VideoCapture("battery.avi")

_, first_frame = video.read() # 영상에서 첫번째 프레임만 얻기 위해
x = 135
y = 270
width = 230
height = 400
roi = first_frame[y: y + height, x: x + width]  # first_frame에서 배터리 이미지만 추출
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) #..?
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
# HSV는 channel이 3개이므로 정확한 측정을 위해서 1개의 channel만 사용
roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX) # 최소 0, 최대 255 설정
print(roi_hist)

term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1) # minshift 알고리즘 세부사항
dc = DepthCamera()

while True:
    ret, depth_frame, frame = dc.get_frame()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    
    _, track_window = cv2.meanShift(mask, (x, y, width, height), term_criteria) # 흰색이 분포하는 곳의 x, y, width, height 좌표출력
    x, y, w, h = track_window # 각각의 변수에 값을 저장
    point = (int((x + w) / 2), int((y + h) / 2))
   
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) # 위에 저장된 변수를 포함하는 사각형 형성 
    cv2.circle(frame, point, 15, (0, 0, 225))
        
    distance = depth_frame[point[1], point[0]]
    cv2.putText(frame, "{}mm".format(distance), (point[0], point[1]), cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 0), 2)
    
    cv2.imshow("Mask", mask) # 대상이미지를 하얀색 점으로 표시 
    cv2.imshow("roi", roi)
    cv2.imshow("first_frame", first_frame)
    cv2.imshow("depth frame", depth_frame)
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1) # battery.avi 영상 재생 속도 설정
    if key == 27:
        break

video.release()
cv2.destroyAllWindows()